# Import libraries

In [ ]:
import os
import gdown
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import scipy as sp
from itertools import product

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit, KFold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_score, confusion_matrix, adjusted_mutual_info_score, adjusted_rand_score
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance_matrix

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError as MAE
from tensorflow.keras.regularizers import L1, L2

# Import utils.py
!gdown 'https://drive.google.com/uc?id=13I5w4WajPg6MObtLPQjxznm8w5hKlEY0' -O ./utils.py
from utils import *

# KIDNEY DATASET

## Load dataset

In [ ]:
# Load dataset
if not os.path.exists("./mRNA.txt"):
    !gdown 'https://drive.google.com/uc?id=1i1do_UTzwXzPVIDDmYSFJEholK2Mp8g_' -O ./mRNA.txt
if not os.path.exists("./miRNA.txt"):
    !gdown 'https://drive.google.com/uc?id=1liKeOBKjnbCi1CIjcOPA3Zxv2fRzCfa2' -O ./miRNA.txt
if not os.path.exists("./meth.txt"):
    !gdown 'https://drive.google.com/uc?id=1qr9joY0bAVDLvjWsKF5xf3CaRolBu-mP' -O ./meth.txt
if not os.path.exists("./clusters.txt"):
    !gdown 'https://drive.google.com/uc?id=1R-U2iDgM4oEyzNRfBIA2kXMbKw_s0QtI' -O ./clusters.txt

ds = {}     # this will contain each omic
omics = ['mRNA','miRNA','meth']
for omic_name in omics:
    path = omic_name + ".txt"
    if omic_name not in ds:
        ds[omic_name] = pd.read_csv(path, sep='\t', index_col=0)

y = pd.read_csv('clusters.txt', sep='\t', index_col=0)
true_cluster_labels = y.values.reshape(y.shape[0])

## Preprocess omics

### mRNA

In [ ]:
# Keep only protein coding genes
# NB: the file idx_mRNA_prot_cod.txt, containing the protein coding genes of the
# human genome, has been obtained in the colab notebook 'Find protein coding genes'
if not os.path.exists("./idx_mRNA_prot_cod_kidney.txt"):
    !gdown 'https://drive.google.com/uc?id=1Pi4u8y_YAc2tmOWZYaeLn9wGdzu4cFC5' -O ./idx_mRNA_prot_cod_kidney.txt

idx_mRNA_prot_cod = pd.read_csv('idx_mRNA_prot_cod_kidney.txt')
idx_mRNA_prot_cod = idx_mRNA_prot_cod['idx'].values

ds['mRNA'] = ds['mRNA'].iloc[:, idx_mRNA_prot_cod]

# Delete genes with a zero expression value across all the samples
ds['mRNA'] = ds['mRNA'].loc[:, (ds['mRNA'] != 0).any(axis=0)]

# Normalize mRNA with MinMax Scaler
ds['mRNA_normalized'] = MinMaxScaler().fit_transform(ds['mRNA'].values)


### miRNA

In [ ]:
# Delete sequences with a zero expression value across all the samples
ds['miRNA'] = ds['miRNA'].loc[:, (ds['miRNA'] != 0).any(axis=0)]

# Normalize with log2 normalization
ds['miRNA'] = np.log(ds['miRNA'] + 1) / np.log(2)

# Normalize with MinMaxScaler
ds['miRNA_normalized'] = MinMaxScaler().fit_transform(ds['miRNA'].values)

### meth

In [ ]:
# Delete sequences with a zero expression value across all the samples
ds['meth'] = ds['meth'].loc[:, (ds['meth'] != 0).any(axis=0)]

# Normalize with MinMaxScaler
ds['meth_normalized'] = MinMaxScaler().fit_transform(ds['meth'].values)

### Visualize each omic with true cluster labels

Each omic is first reduced to two dimensions with PCA, and then the two principal components are plotted in the 2D plane

In [ ]:
for omic in omics:
    print(omic)

    # Perform a 2D PCA to visualize the dataset
    pca = PCA(2)
    principalComponents = pca.fit_transform(ds[f'{omic}_normalized'])

    # Plot the clustered dataset with true cluster labels
    plot_2D_dataset(principalComponents, true_cluster_labels, title=f'{omic} visualization')

    print()
    print()

## Preliminar analysis

### Apply K-means on the original space for each omic

> K-means on the original space of each omic.

> Plot the reduced dataset at 2 dimensions with cluster assignment and with true cluster labels.

> Evaluation of cluster assignments with the confusion matrix.

> Silhouette score computation

In [ ]:
for omic in omics:
    print(omic)

    # Apply K-means
    kmeans = KMeans(n_clusters=3, random_state=0)
    cluster_labels = kmeans.fit_predict(ds[f'{omic}_normalized'])

    # Perform a 2D PCA to visualize the dataset
    pca = PCA(2)
    principalComponents = pca.fit_transform(ds[f'{omic}_normalized'])
    kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

    # Plot the clustered dataset with cluster assignments and true cluster labels
    plot_2D_dataset(principalComponents, cluster_labels, title=f'{omic} visualization', caption='predicted clusters')
    plot_2D_dataset(principalComponents, true_cluster_labels, title=f'{omic} visualization', caption='true clusters')

    # Plot the confusion matrix
    plot_confusion_matrix(true_cluster_labels, cluster_labels)

    # Compute silhouette on the original dataset with cluster assignments and true cluster labels
    print(f"Silhouette, predicted clusters: {silhouette_score(ds[f'{omic}_normalized'], cluster_labels)}")
    print(f"Silhouette, true clusters: {silhouette_score(ds[f'{omic}_normalized'], true_cluster_labels)}")
    
    print()
    print()
    print()



### Apply K-means on the reduced space (64 features) for each omic

> K-means on the reduced space at 64 dimensions space of each omic.

> Plot the reduced dataset at 2 dimensions with cluster assignment and with true cluster labels.

> Evaluation of cluster assignments with the confusion matrix.

> Silhouette score computation

In [ ]:
for omic in omics:
    print(omic)
    # Perform dimensionality reduction --> PCA(64)
    pca = PCA(64)
    ds[f'{omic}_normalized_reduced'] = pca.fit_transform(ds[f'{omic}_normalized'])

    # Apply K-means
    kmeans = KMeans(n_clusters=3, random_state=0)
    cluster_labels = kmeans.fit_predict(ds[f'{omic}_normalized_reduced'])

    # Perform a 2D PCA to visualize the dataset
    pca = PCA(2)
    principalComponents = pca.fit_transform(ds[f'{omic}_normalized_reduced'])
    kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

    # Plot the clustered dataset with cluster assignments and true cluster labels
    plot_2D_dataset(principalComponents, cluster_labels, title=f'{omic} 64D visualization', caption='predicted clusters')
    plot_2D_dataset(principalComponents, true_cluster_labels, title=f'{omic} 64D visualization', caption='true clusters')

    # Plot the confusion matrix
    plot_confusion_matrix(true_cluster_labels, cluster_labels)

    # Compute silhouette on the original dataset with cluster assignments and true cluster labels
    print(f"Silhouette, predicted clusters: {silhouette_score(ds[f'{omic}_normalized_reduced'], cluster_labels)}")
    print(f"Silhouette, true clusters: {silhouette_score(ds[f'{omic}_normalized_reduced'], true_cluster_labels)}")

    print()
    print()
    print()


### Apply K-means on the early integrated dataset

> Concatenation of the dataset for each omic in on early integrated dataset.

> K-means on the early integrated dataset.

> Plot the reduced dataset at 2 dimensions with cluster assignment and with true cluster labels.

> Evaluation of cluster assignments with the confusion matrix.

> Silhouette score computation

In [ ]:
# Concatenate the omics (early integration)
ds['early_integr'] = np.concatenate([ds[f'{omic}_normalized'] for omic in omics], axis=1)

for K in range(2,11):
    # Apply K-means
    kmeans = KMeans(n_clusters=K, random_state=0)
    cluster_labels = kmeans.fit_predict(ds['early_integr'])

    # Perform a 2D PCA to visualize the dataset
    pca = PCA(2)
    principalComponents = pca.fit_transform(ds['early_integr'])
    kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

    # Plot the clustered dataset with cluster assignments and true cluster labels
    plot_2D_dataset(principalComponents, cluster_labels, cluster_centers=kmeans.cluster_centers_, title=f'Early integrated dataset visualization', caption=f'{K} predicted clusters')
    plot_2D_dataset(principalComponents, true_cluster_labels, title=f'Early integrated dataset visualization', caption='true clusters')

    # Plot the confusion matrix
    plot_confusion_matrix(true_cluster_labels, cluster_labels)

    # Compute silhouette on the original dataset with cluster assignments and true cluster labels
    print(f"Silhouette, predicted clusters: {silhouette_score(ds['early_integr'], cluster_labels)}")

### Data Exploration

In [ ]:
def data_exploration(df):
    n_samples, n_features = df.shape
    print(f"##### N. of samples: {n_samples}")
    print(f"##### N. of features: {n_features}")
    print()

    print("##### Last 5 samples of the transcriptome:")
    print(df.tail())
    print()

    print("##### Are there any missing values?")
    print(df.isna().sum().any())
    print()

data_exploration(pd.DataFrame(data= ds['early_integr']))

### Scree plot

Scree plot to determine the number of factors to retain principal components to keep in a principal component analysis (PCA).

In [ ]:
# PCA
def elbow(df):
    pca = PCA().fit(df)
    height = pca.explained_variance_ratio_*100
    y_pos = np.arange(df.values.shape[1])
    if df.values.shape[1] < 500:
        plt.figure(figsize = (30,5))
        plt.bar(y_pos,height)
        plt.xticks(y_pos,np.arange(df.values.shape[1]))
        plt.ylim(0,110)
        plt.plot(np.cumsum(pca.explained_variance_ratio_)*100, 'r-s')
        plt.xlabel('number of components')
        plt.ylabel('cumulative explained variance')
        plt.show()
    else:
        n_principal_components = 200
        plt.figure(figsize = (30,5))
        plt.bar(np.arange(n_principal_components),pca.explained_variance_ratio_[:n_principal_components]*100)
        plt.xticks(y_pos,np.arange(n_principal_components)) #df.values.shape[1]
        plt.ylim(0,110)
        plt.plot(np.cumsum(pca.explained_variance_ratio_[:n_principal_components])*100, 'r-s')
        plt.xlabel('number of components')
        plt.ylabel('cumulative explained variance')
        plt.show()
        """
        print('Too many features! Image too big to be shown.')
        print('Cumulative sum of explained variance:')
        print(np.cumsum(pca.explained_variance_ratio_)*100)
        """
for omic in omics:
    print(omic)
    elbow(pd.DataFrame(ds[f'{omic}_normalized']))
    print()

## Our method

### mRNA

In [ ]:
n_samples, n_features = ds['mRNA'].shape
ds['mRNA'].shape

Grid search

In [ ]:
n_samples, n_features = ds['mRNA_normalized'].shape

##### Hyperparameters
# For the grid search
N_SPLITS = 3
kf = KFold(n_splits=N_SPLITS)
idx = np.arange(len(ds['mRNA_normalized']))   # 0,1,...,783; indices used for the K-fold splitting of the omic

bs_param = [16,32]
lr_param = [0.0005,0.0001]

# For the training
EPOCHS_PHASE_1 = 100    # max n. of epochs for the 1st phase (if no early stopping is taken)
EPOCHS_PHASE_2 = 60     # max n. of epochs for the 2nd phase (if no early stopping is taken)
N_EPOCHS = EPOCHS_PHASE_1 + EPOCHS_PHASE_2  # max n. of epochs (N_EPOCHS are performed if no early stopping is taken)
MIN_DELTA = 0.005   # threshold for the early stopping
PATIENCE = 30       # threshold for the early stopping
MAX_CLUSTERS = 10   # max n. of clusters to search for at the end of the 1st phase
LMBD = 0.1          # weight of the closest cluster center loss when it is summed to the reconstruction loss

optimal_val_rec_loss = np.inf   # this variable will record the best validation reconstruction loss obtained at the end of the grid search

for BATCH_SIZE, LR in cartesian(bs_param, lr_param):
    print('Training with:')
    print(f'Batch size: {BATCH_SIZE}')
    print(f'Learning Rate: {LR}')
    EARLY_STOPPING_PHASE_1 = []     # will store the N_SPLITS values obtained for the n. of epochs of phase 1 performed
    EARLY_STOPPING_PHASE_2 = []

    for train_index, test_index in kf.split(idx):
        # N_SPLITS-1 training splits, 1 validation split
        X_train, X_test = ds['mRNA_normalized'][train_index], ds['mRNA_normalized'][test_index]
        Y_train = true_cluster_labels[train_index]

        # Shuffle and split the training set in minibatches
        tensor_train_ds = tf.convert_to_tensor(X_train, dtype=tf.float32)    # cast to float32 (altrimenti non funziona, per un motivo a me ignoto)
        temp_ds = Dataset.from_tensor_slices(tensor_train_ds)
        shuffle_ds = temp_ds.shuffle(buffer_size=ds['mRNA'].shape[0], reshuffle_each_iteration=True)
        batch_ds = shuffle_ds.batch(BATCH_SIZE)

        # Build the validation set (no batches)
        tensor_val_ds = tf.convert_to_tensor(X_test, dtype=tf.float32)    # cast to float32 (altrimenti non funziona, per un motivo a me ignoto)

        

        #####################################################################################
        ##############################  MODEL ARCHITECTURE  #################################
        #####################################################################################

        ##### Autoencoder architecture
        input_window = Input(shape=n_features)

        # "encoded" is the encoded representation of the input
        x = Dense(1945, activation='softsign', name='encoder_1')(input_window)   # 2048
        x = BatchNormalization()(x)
        x = Dense(194, activation='softsign', name='encoder_2')(x)   # 2048
        x = BatchNormalization()(x)
        encoded_layer = Dense(64, activation='sigmoid', name='encoder_3')(x)

        # "decoded" is the lossy reconstruction of the input
        x = Dense(194, activation='softsign', name='decoder_1')(encoded_layer)
        x = BatchNormalization()(x)
        x = Dense(1945, activation='softsign', name='decoder_2')(encoded_layer)
        x = BatchNormalization()(x)
        decoded_layer = Dense(n_features, activation='sigmoid', name='decoder_3')(x)

        # this model maps an input to its reconstruction and to its encoded representation in the latent space
        autoencoder = Model(input_window, outputs = [decoded_layer, encoded_layer])

        # this model maps an input to its encoded representation in the latent space
        encoder = Model(input_window, encoded_layer)  # restituisce solo il sample encoded

        print(autoencoder.summary())



        #####################################################################################
        ###################################  TRAINING  ######################################
        #####################################################################################
        
        train_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample loss (the mean is taken for each minibatch)
        train_rec_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample reconstruction loss
        train_ccc_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample closest cluster center loss

        val_rec_loss = np.zeros(N_EPOCHS)   # mean per-sample reconstruction loss
        
        optimizer = Adam(learning_rate = LR)





        ##### TRAINING PHASE 1 #####

        for epoch in range(1, EPOCHS_PHASE_1+1):  # eventual early stopping
            print(f"\nStart of epoch {epoch}")
            
            # Iterate over the batches of the dataset.
            for step, batch in enumerate(batch_ds):

                # Open a GradientTape to record the operations run
                # during the forward pass, which enables auto-differentiation.
                with tf.GradientTape() as tape:

                    # Run the forward pass of the layer.
                    # The operations that the layer applies
                    # to its inputs are going to be recorded
                    # on the GradientTape.
                    decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (in the latent space)

                    # Compute the loss value for this minibatch.
                    loss_value = custom_loss(batch, decoded_batch, phase=1)
                    
                    train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
                    train_rec_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)

                # Use the gradient tape to automatically retrieve
                # the gradients of the trainable variables with respect to the loss.
                grads = tape.gradient(loss_value, autoencoder.trainable_weights)

                # Run one step of gradient descent by updating
                # the value of the variables to minimize the loss.
                optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

            print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))


            # End of the epoch: VALIDATION STEP
            decoded_val_ds, _ = autoencoder(tensor_val_ds, training=False)  # Logits for this minibatch (in the latent space)
            loss_value_val = custom_loss(tensor_val_ds, decoded_val_ds, phase=1)    # mean per-sample validation reconstruction loss
            val_rec_loss[epoch-1] = tf.cast(loss_value_val, tf.float32)
            print("Mean validation loss for epoch %d = %.4f" % (epoch, val_rec_loss[epoch-1]))

            # Check if early stopping conditions are met
            stopEarly = Callback_EarlyStopping(val_rec_loss[:epoch-1], min_delta=MIN_DELTA, patience=PATIENCE)
            if stopEarly:
                print("Callback_EarlyStopping signal received at epoch= %d/%d"%(epoch,EPOCHS_PHASE_1))
                print("Terminating training")           
                break
        EARLY_STOPPING_PHASE_1.append(epoch) # populate vector of number of epochsof phase 1
        last_epoch_phase_1 = epoch  # less than or equal to EPOCHS_PHASE_1
        





        ##### Find the n. of clusters with kmeans in the latent space     
        ds['mRNA_encoded'] = encoder(X_train)   # (n_samples, ls_dim)
        opt_silhouette = -1 

        for n_clusters in np.arange(2, MAX_CLUSTERS+1):
            kmeans = KMeans(n_clusters=n_clusters, random_state=0)
            cluster_assignments = kmeans.fit_predict(ds['mRNA_encoded'])
            silhouette_avg = silhouette_score(ds['mRNA_encoded'], cluster_assignments)

            # Find the optimal number number of clusters, based on silhouette score
            if silhouette_avg >= opt_silhouette:    # better n. of clusters found
                n_clust_opt = n_clusters
                opt_silhouette = silhouette_avg

        print(f"The best number of clusters found is: {n_clust_opt}")
        n_clusters = n_clust_opt
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        cluster_assignments = kmeans.fit_predict(ds['mRNA_encoded'])

        # Perform a 2D PCA to visualize the dataset
        pca = PCA(2)
        pc2 = pca.fit_transform(ds['mRNA_encoded'])
        # Cluster centers in the latent space (needed for computing the loss in phase 2)
        cluster_centers = kmeans.cluster_centers_
        # Reduce cluster centers dimensions (for visualization)
        kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

        # Plot 2D visualization of the encoded dataset with the cluster assignments found
        plot_2D_dataset(pc2, cluster_assignments, cluster_centers=kmeans.cluster_centers_, title='Encoded mRNA dataset')
        print(f"Number of clusters: {cluster_centers.shape[0]}")

        # Compute silhouette score in the latent space
        silhouette_avg = silhouette_score(ds['mRNA_encoded'], cluster_assignments)
        print(f"Silhouette on the encoded dataset: {silhouette_avg}")







        ##### TRAINING PHASE 2 #####

        for epoch in range(last_epoch_phase_1 + 1, last_epoch_phase_1 + EPOCHS_PHASE_2 + 1):  # eventual early stopping
            print(f"\nStart of epoch {epoch}")

            # Iterate over the batches of the dataset.
            for step, batch in enumerate(batch_ds):

                # Open a GradientTape to record the operations run
                # during the forward pass, which enables auto-differentiation.
                with tf.GradientTape() as tape:

                    # Run the forward pass of the layer.
                    # The operations that the layer applies
                    # to its inputs are going to be recorded
                    # on the GradientTape.
                    decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (in the latent space)

                    # Compute the loss value for this minibatch.
                    # To compute this loss we need the cluster centers of the previous step (t-1)
                    loss_value, l1, l2 = custom_loss(batch, decoded_batch, encoded_batch, cluster_centers, lmbd=LMBD, phase=2)
                    
                    train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
                    train_rec_loss[epoch-1][step] = l1
                    train_ccc_loss[epoch-1][step] = l2

                # Use the gradient tape to automatically retrieve
                # the gradients of the trainable variables with respect to the loss.
                grads = tape.gradient(loss_value, autoencoder.trainable_weights)

                # Run one step of gradient descent by updating
                # the value of the variables to minimize the loss.
                optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

            # The autoencoder is now updated
            # Update the K-means cluster assignments and cluster centers
            ds['mRNA_encoded'] = encoder(X_train)
            kmeans = KMeans(n_clusters=n_clusters, random_state=0)
            cluster_assignments_new = kmeans.fit_predict(ds['mRNA_encoded'])
            cluster_centers_new = kmeans.cluster_centers_

            # Evaluate the silhouette w.r.t. the new cluster assignments:
            # if the encoded dataset is more separable with these assignments,
            # cluster assignment and cluster centers are updated, otherwise they remain unchanged
            if silhouette_avg < silhouette_score(ds['mRNA_encoded'], cluster_assignments_new):
                cluster_assignments = cluster_assignments_new
                cluster_centers = cluster_centers_new
                print("Cluster centers have changed")
                    
            print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
            print("Mean training reconstruction loss for epoch %d = %.4f" % (epoch, train_rec_loss[epoch-1].mean()))
            print("Mean training closest cluster loss for epoch %d = %.4f" % (epoch, train_ccc_loss[epoch-1].mean()))
            
            silhouette_avg = silhouette_score(ds['mRNA_encoded'], cluster_assignments)
            print(f"Silhouette on the encoded dataset: {silhouette_avg}")
            
            # Perform a 2D PCA to visualize the dataset
            pca = PCA(2)
            pc2 = pca.fit_transform(ds['mRNA_encoded'])
            # Plot the encoded space in 2D with predicted clusters and true cluster labels
            plot_2D_dataset(pc2, cluster_assignments, title='Encoded mRNA', caption='predicted clusters')
            plot_2D_dataset(pc2, Y_train, title='Encoded mRNA', caption='true clusters')
            # Plot confusion matrix
            plot_confusion_matrix(Y_train, cluster_assignments)


            # End of the epoch: VALIDATION STEP
            decoded_val_ds, _ = autoencoder(tensor_val_ds, training=False)  # Logits for this minibatch (in the latent space)
            loss_value_val = custom_loss(tensor_val_ds, decoded_val_ds, encoded_batch=None, phase=1)
            val_rec_loss[epoch-1] = tf.cast(loss_value_val, tf.float32)
            print("Mean validation loss for epoch %d = %.4f" % (epoch, val_rec_loss[epoch-1]))

            # Check if early stopping conditions are met
            stopEarly = Callback_EarlyStopping(val_rec_loss[EPOCHS_PHASE_1:epoch-1], min_delta=MIN_DELTA, patience=PATIENCE)
            if stopEarly:
                print("Callback_EarlyStopping signal received at epoch= %d/%d"%(epoch,EPOCHS_PHASE_1))
                print("Terminating training")
                break
        EARLY_STOPPING_PHASE_2.append(epoch - last_epoch_phase_1)   # populate vector of number of epochs of phase 2
    
    # Remove the zero values which have not been modified (because of early stopping)
    train_loss = train_loss.mean(axis=1)[:epoch-1]
    train_rec_loss = train_rec_loss.mean(axis=1)[:epoch-1]
    train_ccc_loss = train_ccc_loss.mean(axis=1)[:epoch-1]
    val_rec_loss = val_rec_loss[:epoch-1]

    plot_confusion_matrix(Y_train, cluster_assignments)

    # Evolution of training and validation losses
    plt.plot(np.arange(1,len(train_loss)+1), train_loss)
    plt.title('Train loss')
    plt.show()

    plt.plot(np.arange(1,len(train_rec_loss)+1), train_rec_loss)
    plt.title('Train reconstruction loss')
    plt.show()

    plt.plot(np.arange(1,len(train_ccc_loss)+1), train_ccc_loss)
    plt.title('Train closest cluster center loss')
    plt.show()

    plt.plot(np.arange(1,len(val_rec_loss)+1), val_rec_loss)
    plt.title('Validation reconstruction loss')
    plt.show()

    # For the grid search: are the current hyperparameters better than the others tested so far?
    if val_rec_loss[-1] < optimal_val_rec_loss:
        optimal_val_rec_loss = val_rec_loss[-1]
        BEST_BATCH_SIZE = BATCH_SIZE
        BEST_LR = LR
        BEST_EPOCH_PHASE_1 = int(np.trunc(np.mean(EARLY_STOPPING_PHASE_1))) # mean value of the epochs of the 1st phase (the average is across the N_SPLITS values obtained during cross-val)
        BEST_EPOCH_PHASE_2 = int(np.trunc(np.mean(EARLY_STOPPING_PHASE_2))) # mean value of the epochs of the 2nd phase (the average is across the N_SPLITS values obtained during cross-val)


print(f'\nBest params found: Batch size: {BEST_BATCH_SIZE}, Learning Rate: {BEST_LR}, Epochs 1st phase: {BEST_EPOCH_PHASE_1}, Epochs 2nd phase: {BEST_EPOCH_PHASE_2}')

BEST_BATCH_SIZE = 16

BEST_LR = 0.0005

BEST_EPOCH_PHASE_1 =  100

BEST_EPOCH_PHASE_2 = 60 

Training

In [ ]:
# Hyperparameters
BATCH_SIZE = BEST_BATCH_SIZE
LR = BEST_LR
N_EPOCHS = BEST_EPOCH_PHASE_1 + BEST_EPOCH_PHASE_2
EPOCHS_PHASE_1 = BEST_EPOCH_PHASE_1
MAX_CLUSTERS = 10   # max n. of clusters to search for at the end of the 1st phase
LMBD = 0.1 # 0.1


# Shuffle and split the dataset in minibatches
tensor_train_ds = tf.convert_to_tensor(ds['mRNA_normalized'], dtype=tf.float32)
temp_ds = Dataset.from_tensor_slices(tensor_train_ds)
shuffle_ds = temp_ds.shuffle(buffer_size=ds['mRNA'].shape[0], reshuffle_each_iteration=True)
batch_ds = shuffle_ds.batch(BATCH_SIZE)

#####################################################################################
##############################  MODEL ARCHITECTURE  #################################
#####################################################################################

##### Autoencoder architecture
# this is our input placeholder (cioè ogni sample in input deve avere n_features features)
input_window = Input(shape=n_features)

# "encoded" is the encoded representation of the input
x = Dense(1945, activation='softsign', name='encoder_1')(input_window)
x = BatchNormalization()(x)
x = Dense(194, activation='softsign', name='encoder_2')(x)
x = BatchNormalization()(x)
encoded_layer = Dense(64, activation='sigmoid', name='encoder_3')(x)

# TODO vd pag. 2 del pdf: come output dell'encoder lui mette una funzione sigmoide

# "decoded" is the lossy reconstruction of the input
x = Dense(194, activation='softsign', name='decoder_1')(encoded_layer)
x = BatchNormalization()(x)
x = Dense(1945, activation='softsign', name='decoder_2')(x)
x = BatchNormalization()(x)
decoded_layer = Dense(n_features, activation='sigmoid', name='decoder_3')(x)

# this model maps an input to its reconstruction
autoencoder = Model(input_window, outputs = [decoded_layer,encoded_layer])  # quando è chiamato autoencoder.predict(), ritorna sia il sample encoded che quello decoded

# this model maps an input to its encoded representation in the latent space
encoder = Model(input_window, encoded_layer)  # restituisce solo il sample encoded

print(autoencoder.summary())


#####################################################################################
###################################  TRAINING  ######################################
#####################################################################################

train_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample loss (the mean is taken for each minibatch)
train_rec_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample reconstruction loss
train_ccc_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample closest cluster center loss

optimizer = Adam(LR)



##### TRAINING PHASE 1 #####
for epoch in range(1, BEST_EPOCH_PHASE_1+1): 
    print(f"\nStart of epoch {epoch}")

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(batch_ds):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (nel latent space)

            # Compute the loss value for this minibatch
            loss_value = custom_loss(batch, decoded_batch, phase=1)
            
            train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
            train_rec_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, autoencoder.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

    print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
##### END OF TRAINING PHASE 1 #####
        


##### Find the n. of clusters with kmeans in the latent space
ds['mRNA_encoded'] = encoder(ds['mRNA_normalized'])   # (n_samples, ls_dim)
opt_silhouette = -1 

for n_clusters in np.arange(2, MAX_CLUSTERS+1):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_assignments = kmeans.fit_predict(ds['mRNA_encoded'])
    silhouette_avg = silhouette_score(ds['mRNA_encoded'], cluster_assignments)

    # Find the optimal number number of clusters, based on silhouette score
    if silhouette_avg >= opt_silhouette:    # better n. of clusters found
        n_clust_opt = n_clusters
        opt_silhouette = silhouette_avg

print(f"The best number of clusters found is: {n_clust_opt}")
n_clusters = n_clust_opt
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
cluster_assignments = kmeans.fit_predict(ds['mRNA_encoded'])

# Perform a 2D PCA to visualize the dataset
pca = PCA(2)
pc2 = pca.fit_transform(ds['mRNA_encoded'])
# Cluster centers in the latent space (needed for computing the loss in phase 2)
cluster_centers = kmeans.cluster_centers_
# Reduce cluster centers dimensions (for visualization)
kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

# Plot 2D visualization of the encoded dataset with the cluster assignments found
plot_2D_dataset(pc2, cluster_assignments, cluster_centers=kmeans.cluster_centers_, title='Encoded mRNA dataset')
print(f"Number of clusters: {cluster_centers.shape[0]}")

# Compute silhouette score in the latent space
silhouette_avg = silhouette_score(ds['mRNA_encoded'], cluster_assignments)
print(f"Silhouette on the encoded dataset: {silhouette_avg}")


       
##### TRAINING PHASE 2 #####
for epoch in range(BEST_EPOCH_PHASE_1 + 1, N_EPOCHS + 1):  # eventual early stopping
    print(f"\nStart of epoch {epoch}")

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(batch_ds):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (nel latent space)

            # Compute the loss value for this minibatch.
            # To compute this loss we need the cluster centers of the previous step (t-1)

            loss_value, l1, l2 = custom_loss(batch, decoded_batch, encoded_batch, cluster_centers, lmbd=LMBD, phase=2)
            
            train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
            train_rec_loss[epoch-1][step] = l1
            train_ccc_loss[epoch-1][step] = l2

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, autoencoder.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))


    ds['mRNA_encoded'] = encoder(ds['mRNA_normalized']) 

    # Perform KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_assignments_new = kmeans.fit_predict(ds['mRNA_encoded'])
    cluster_centers_new = np.copy(kmeans.cluster_centers_)

    if silhouette_avg < silhouette_score(ds['mRNA_encoded'], cluster_assignments_new):
        cluster_assignments = cluster_assignments_new
        cluster_centers = cluster_centers_new
        print("Cluster centers have changed")
            
    print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
    print("Mean training reconstruction loss for epoch %d = %.4f" % (epoch, train_rec_loss[epoch-1].mean()))
    print("Mean training closest cluster loss for epoch %d = %.4f" % (epoch, train_ccc_loss[epoch-1].mean()))
                
    silhouette_avg = silhouette_score(ds['mRNA_encoded'], cluster_assignments)
    print(f"Silhouette on the encoded dataset: {silhouette_avg}")

    # Perform a 2D PCA to visualize the dataset
    pca2 = PCA(2)
    pc2 = pca2.fit_transform(ds['mRNA_encoded'])
    # Plot the encoded space in 2D with predicted clusters and true cluster labels
    plot_2D_dataset(pc2, cluster_assignments, title='Encoded mRNA', caption='predicted clusters')
    plot_2D_dataset(pc2, true_cluster_labels, title='Encoded mRNA', caption='true clusters')

    # Plot confusion matrix
    plot_confusion_matrix(true_cluster_labels, cluster_assignments)
##### END OF TRAINING PHASE 2 #####

train_loss = train_loss.mean(axis=1)[:epoch-1]
train_rec_loss = train_rec_loss.mean(axis=1)[:epoch-1]
train_ccc_loss = train_ccc_loss.mean(axis=1)[:epoch-1]

# Evolution of training and validation losses
plt.plot(np.arange(1,len(train_loss)+1), train_loss)
plt.title('Train loss')
plt.show()

plt.plot(np.arange(1,len(train_rec_loss)+1), train_rec_loss)
plt.title('Train reconstruction loss')
plt.show()

plt.plot(np.arange(1,len(train_ccc_loss)+1), train_ccc_loss)
plt.title('Train closest cluster center loss')
plt.show()

# Save the autoencoder
autoencoder.save('autoencoder_mRNA_kidney.tf')


### miRNA

In [ ]:
n_samples, n_features = ds['miRNA'].shape
ds['miRNA'].shape

Grid search

In [ ]:
n_samples, n_features = ds['miRNA_normalized'].shape

##### Hyperparameters
# For the grid search
N_SPLITS = 3
kf = KFold(n_splits=N_SPLITS)
idx = np.arange(len(ds['miRNA_normalized']))   # 0,1,...,783; indices used for the K-fold splitting of the omic

bs_param = [16,32] # [16,32]
lr_param = [0.005,0.001] # [0.005,0.001]

# For the training
EPOCHS_PHASE_1 = 100    # max n. of epochs for the 1st phase (if no early stopping is taken)
EPOCHS_PHASE_2 = 60     # max n. of epochs for the 2nd phase (if no early stopping is taken)
N_EPOCHS = EPOCHS_PHASE_1 + EPOCHS_PHASE_2  # max n. of epochs (N_EPOCHS are performed if no early stopping is taken)
MIN_DELTA = 0.005   # threshold for the early stopping
PATIENCE = 30       # threshold for the early stopping
MAX_CLUSTERS = 10   # max n. of clusters to search for at the end of the 1st phase
LMBD = 0.1          # weight of the closest cluster center loss when it is summed to the reconstruction loss

optimal_val_rec_loss = np.inf   # this variable will record the best validation reconstruction loss obtained at the end of the grid search

for BATCH_SIZE, LR in cartesian(bs_param, lr_param):
    print('Training with:')
    print(f'Batch size: {BATCH_SIZE}')
    print(f'Learning Rate: {LR}')
    EARLY_STOPPING_PHASE_1 = []     # will store the N_SPLITS values obtained for the n. of epochs of phase 1 performed
    EARLY_STOPPING_PHASE_2 = []

    for train_index, test_index in kf.split(idx):
        # N_SPLITS-1 training splits, 1 validation split
        X_train, X_test = ds['miRNA_normalized'][train_index], ds['miRNA_normalized'][test_index]
        Y_train = true_cluster_labels[train_index]

        # Shuffle and split the training set in minibatches
        tensor_train_ds = tf.convert_to_tensor(X_train, dtype=tf.float32)    # cast to float32 (altrimenti non funziona, per un motivo a me ignoto)
        temp_ds = Dataset.from_tensor_slices(tensor_train_ds)
        shuffle_ds = temp_ds.shuffle(buffer_size=ds['miRNA'].shape[0], reshuffle_each_iteration=True)
        batch_ds = shuffle_ds.batch(BATCH_SIZE)

        # Build the validation set (no batches)
        tensor_val_ds = tf.convert_to_tensor(X_test, dtype=tf.float32)    # cast to float32 (altrimenti non funziona, per un motivo a me ignoto)

        

        #####################################################################################
        ##############################  MODEL ARCHITECTURE  #################################
        #####################################################################################

        ##### Autoencoder architecture
        input_window = Input(shape=n_features)

        # "encoded" is the encoded representation of the input
        x = Dense(162, activation='softsign', name='encoder_1')(input_window)
        x = BatchNormalization()(x)
        x = Dense(100, activation='softsign', name='encoder_2')(x)
        x = BatchNormalization()(x)
        encoded_layer = Dense(64, activation='sigmoid', name='encoder_3')(x)

        # "decoded" is the lossy reconstruction of the input
        x = Dense(100, activation='softsign', name='decoder_1')(encoded_layer)
        x = BatchNormalization()(x)
        x = Dense(162, activation='softsign', name='decoder_2')(encoded_layer)
        x = BatchNormalization()(x)
        decoded_layer = Dense(n_features, activation='sigmoid', name='decoder_3')(x)

        # this model maps an input to its reconstruction and to its encoded representation in the latent space
        autoencoder = Model(input_window, outputs = [decoded_layer, encoded_layer])

        # this model maps an input to its encoded representation in the latent space
        encoder = Model(input_window, encoded_layer)  # restituisce solo il sample encoded

        print(autoencoder.summary())



        #####################################################################################
        ###################################  TRAINING  ######################################
        #####################################################################################
        
        train_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample loss (the mean is taken for each minibatch)
        train_rec_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample reconstruction loss
        train_ccc_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample closest cluster center loss

        val_rec_loss = np.zeros(N_EPOCHS)   # mean per-sample reconstruction loss
        
        optimizer = Adam(learning_rate = LR)





        ##### TRAINING PHASE 1 #####

        for epoch in range(1, EPOCHS_PHASE_1+1):  # eventual early stopping
            print(f"\nStart of epoch {epoch}")
            
            # Iterate over the batches of the dataset.
            for step, batch in enumerate(batch_ds):

                # Open a GradientTape to record the operations run
                # during the forward pass, which enables auto-differentiation.
                with tf.GradientTape() as tape:

                    # Run the forward pass of the layer.
                    # The operations that the layer applies
                    # to its inputs are going to be recorded
                    # on the GradientTape.
                    decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (in the latent space)

                    # Compute the loss value for this minibatch.
                    loss_value = custom_loss(batch, decoded_batch, phase=1)
                    
                    train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
                    train_rec_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)

                # Use the gradient tape to automatically retrieve
                # the gradients of the trainable variables with respect to the loss.
                grads = tape.gradient(loss_value, autoencoder.trainable_weights)

                # Run one step of gradient descent by updating
                # the value of the variables to minimize the loss.
                optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

            print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))


            # End of the epoch: VALIDATION STEP
            decoded_val_ds, _ = autoencoder(tensor_val_ds, training=False)  # Logits for this minibatch (in the latent space)
            loss_value_val = custom_loss(tensor_val_ds, decoded_val_ds, phase=1)    # mean per-sample validation reconstruction loss
            val_rec_loss[epoch-1] = tf.cast(loss_value_val, tf.float32)
            print("Mean validation loss for epoch %d = %.4f" % (epoch, val_rec_loss[epoch-1]))

            # Check if early stopping conditions are met
            stopEarly = Callback_EarlyStopping(val_rec_loss[:epoch-1], min_delta=MIN_DELTA, patience=PATIENCE)
            if stopEarly:
                print("Callback_EarlyStopping signal received at epoch= %d/%d"%(epoch,EPOCHS_PHASE_1))
                print("Terminating training")           
                break
        EARLY_STOPPING_PHASE_1.append(epoch) # populate vector of number of epochsof phase 1
        last_epoch_phase_1 = epoch  # less than or equal to EPOCHS_PHASE_1
        





        ##### Find the n. of clusters with kmeans in the latent space     
        ds['miRNA_encoded'] = encoder(X_train)   # (n_samples, ls_dim)
        opt_silhouette = -1 

        for n_clusters in np.arange(2, MAX_CLUSTERS+1):
            kmeans = KMeans(n_clusters=n_clusters, random_state=0)
            cluster_assignments = kmeans.fit_predict(ds['miRNA_encoded'])
            silhouette_avg = silhouette_score(ds['miRNA_encoded'], cluster_assignments)

            # Find the optimal number number of clusters, based on silhouette score
            if silhouette_avg >= opt_silhouette:    # better n. of clusters found
                n_clust_opt = n_clusters
                opt_silhouette = silhouette_avg

        print(f"The best number of clusters found is: {n_clust_opt}")
        n_clusters = n_clust_opt
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        cluster_assignments = kmeans.fit_predict(ds['miRNA_encoded'])

        # Perform a 2D PCA to visualize the dataset
        pca = PCA(2)
        pc2 = pca.fit_transform(ds['miRNA_encoded'])
        # Cluster centers in the latent space (needed for computing the loss in phase 2)
        cluster_centers = kmeans.cluster_centers_
        # Reduce cluster centers dimensions (for visualization)
        kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

        # Plot 2D visualization of the encoded dataset with the cluster assignments found
        plot_2D_dataset(pc2, cluster_assignments, cluster_centers=kmeans.cluster_centers_, title='Encoded miRNA dataset')
        print(f"Number of clusters: {cluster_centers.shape[0]}")

        # Compute silhouette score in the latent space
        silhouette_avg = silhouette_score(ds['miRNA_encoded'], cluster_assignments)
        print(f"Silhouette on the encoded dataset: {silhouette_avg}")







        ##### TRAINING PHASE 2 #####

        for epoch in range(last_epoch_phase_1 + 1, last_epoch_phase_1 + EPOCHS_PHASE_2 + 1):  # eventual early stopping
            print(f"\nStart of epoch {epoch}")

            # Iterate over the batches of the dataset.
            for step, batch in enumerate(batch_ds):

                # Open a GradientTape to record the operations run
                # during the forward pass, which enables auto-differentiation.
                with tf.GradientTape() as tape:

                    # Run the forward pass of the layer.
                    # The operations that the layer applies
                    # to its inputs are going to be recorded
                    # on the GradientTape.
                    decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (in the latent space)

                    # Compute the loss value for this minibatch.
                    # To compute this loss we need the cluster centers of the previous step (t-1)
                    loss_value, l1, l2 = custom_loss(batch, decoded_batch, encoded_batch, cluster_centers, lmbd=LMBD, phase=2)
                    
                    train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
                    train_rec_loss[epoch-1][step] = l1
                    train_ccc_loss[epoch-1][step] = l2

                # Use the gradient tape to automatically retrieve
                # the gradients of the trainable variables with respect to the loss.
                grads = tape.gradient(loss_value, autoencoder.trainable_weights)

                # Run one step of gradient descent by updating
                # the value of the variables to minimize the loss.
                optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

            # The autoencoder is now updated
            # Update the K-means cluster assignments and cluster centers
            ds['miRNA_encoded'] = encoder(X_train)
            kmeans = KMeans(n_clusters=n_clusters, random_state=0)
            cluster_assignments_new = kmeans.fit_predict(ds['miRNA_encoded'])
            cluster_centers_new = kmeans.cluster_centers_

            # Evaluate the silhouette w.r.t. the new cluster assignments:
            # if the encoded dataset is more separable with these assignments,
            # cluster assignment and cluster centers are updated, otherwise they remain unchanged
            if silhouette_avg < silhouette_score(ds['miRNA_encoded'], cluster_assignments_new):
                cluster_assignments = cluster_assignments_new
                cluster_centers = cluster_centers_new
                print("Cluster centers have changed")
                    
            print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
            print("Mean training reconstruction loss for epoch %d = %.4f" % (epoch, train_rec_loss[epoch-1].mean()))
            print("Mean training closest cluster loss for epoch %d = %.4f" % (epoch, train_ccc_loss[epoch-1].mean()))
            
            silhouette_avg = silhouette_score(ds['miRNA_encoded'], cluster_assignments)
            print(f"Silhouette on the encoded dataset: {silhouette_avg}")
            
            # Perform a 2D PCA to visualize the dataset
            pca = PCA(2)
            pc2 = pca.fit_transform(ds['miRNA_encoded'])
            # Plot the encoded space in 2D with predicted clusters and true cluster labels
            plot_2D_dataset(pc2, cluster_assignments, title='Encoded miRNA', caption='predicted clusters')
            plot_2D_dataset(pc2, Y_train, title='Encoded miRNA', caption='true clusters')
            # Plot confusion matrix
            plot_confusion_matrix(Y_train, cluster_assignments)


            # End of the epoch: VALIDATION STEP
            decoded_val_ds, _ = autoencoder(tensor_val_ds, training=False)  # Logits for this minibatch (in the latent space)
            loss_value_val = custom_loss(tensor_val_ds, decoded_val_ds, encoded_batch=None, phase=1)
            val_rec_loss[epoch-1] = tf.cast(loss_value_val, tf.float32)
            print("Mean validation loss for epoch %d = %.4f" % (epoch, val_rec_loss[epoch-1]))

            # Check if early stopping conditions are met
            stopEarly = Callback_EarlyStopping(val_rec_loss[EPOCHS_PHASE_1:epoch-1], min_delta=MIN_DELTA, patience=PATIENCE)
            if stopEarly:
                print("Callback_EarlyStopping signal received at epoch= %d/%d"%(epoch,EPOCHS_PHASE_1))
                print("Terminating training")
                break
        EARLY_STOPPING_PHASE_2.append(epoch - last_epoch_phase_1)   # populate vector of number of epochs of phase 2
    
    # Remove the zero values which have not been modified (because of early stopping)
    train_loss = train_loss.mean(axis=1)[:epoch-1]
    train_rec_loss = train_rec_loss.mean(axis=1)[:epoch-1]
    train_ccc_loss = train_ccc_loss.mean(axis=1)[:epoch-1]
    val_rec_loss = val_rec_loss[:epoch-1]

    plot_confusion_matrix(Y_train, cluster_assignments)

    # Evolution of training and validation losses
    plt.plot(np.arange(1,len(train_loss)+1), train_loss)
    plt.title('Train loss')
    plt.show()

    plt.plot(np.arange(1,len(train_rec_loss)+1), train_rec_loss)
    plt.title('Train reconstruction loss')
    plt.show()

    plt.plot(np.arange(1,len(train_ccc_loss)+1), train_ccc_loss)
    plt.title('Train closest cluster center loss')
    plt.show()

    plt.plot(np.arange(1,len(val_rec_loss)+1), val_rec_loss)
    plt.title('Validation reconstruction loss')
    plt.show()

    # For the grid search: are the current hyperparameters better than the others tested so far?
    if val_rec_loss[-1] < optimal_val_rec_loss:
        optimal_val_rec_loss = val_rec_loss[-1]
        BEST_BATCH_SIZE = BATCH_SIZE
        BEST_LR = LR
        BEST_EPOCH_PHASE_1 = int(np.trunc(np.mean(EARLY_STOPPING_PHASE_1))) # mean value of the epochs of the 1st phase (the average is across the N_SPLITS values obtained during cross-val)
        BEST_EPOCH_PHASE_2 = int(np.trunc(np.mean(EARLY_STOPPING_PHASE_2))) # mean value of the epochs of the 2nd phase (the average is across the N_SPLITS values obtained during cross-val)


print(f'\nBest params found: Batch size: {BEST_BATCH_SIZE}, Learning Rate: {BEST_LR}, Epochs 1st phase: {BEST_EPOCH_PHASE_1}, Epochs 2nd phase: {BEST_EPOCH_PHASE_2}')

BEST_BATCH_SIZE = 16

BEST_LR = 0.001

BEST_EPOCH_PHASE_1 = 100

BEST_EPOCH_PHASE_2 = 60 

Training

In [ ]:
# Hyperparameters
BATCH_SIZE = BEST_BATCH_SIZE
LR = BEST_LR
N_EPOCHS = BEST_EPOCH_PHASE_1 + BEST_EPOCH_PHASE_2
EPOCHS_PHASE_1 = BEST_EPOCH_PHASE_1
MAX_CLUSTERS = 10   # max n. of clusters to search for at the end of the 1st phase
LMBD = 0.1 # 0.1


# Shuffle and split the dataset in minibatches
tensor_train_ds = tf.convert_to_tensor(ds['miRNA_normalized'], dtype=tf.float32)
temp_ds = Dataset.from_tensor_slices(tensor_train_ds)
shuffle_ds = temp_ds.shuffle(buffer_size=ds['miRNA'].shape[0], reshuffle_each_iteration=True)
batch_ds = shuffle_ds.batch(BATCH_SIZE)

#####################################################################################
##############################  MODEL ARCHITECTURE  #################################
#####################################################################################

##### Autoencoder architecture
# this is our input placeholder (cioè ogni sample in input deve avere n_features features)
input_window = Input(shape=n_features)

# "encoded" is the encoded representation of the input
x = Dense(162, activation='softsign', name='encoder_1')(input_window)
x = BatchNormalization()(x)
x = Dense(100, activation='softsign', name='encoder_2')(x)
x = BatchNormalization()(x)
encoded_layer = Dense(64, activation='sigmoid', name='encoder_3')(x)

# "decoded" is the lossy reconstruction of the input
x = Dense(100, activation='softsign', name='decoder_1')(encoded_layer)
x = BatchNormalization()(x)
x = Dense(162, activation='softsign', name='decoder_2')(encoded_layer)
x = BatchNormalization()(x)
decoded_layer = Dense(n_features, activation='sigmoid', name='decoder_3')(x)

# this model maps an input to its reconstruction
autoencoder = Model(input_window, outputs = [decoded_layer,encoded_layer])  # quando è chiamato autoencoder.predict(), ritorna sia il sample encoded che quello decoded

# this model maps an input to its encoded representation in the latent space
encoder = Model(input_window, encoded_layer)  # restituisce solo il sample encoded

print(autoencoder.summary())


#####################################################################################
###################################  TRAINING  ######################################
#####################################################################################

train_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample loss (the mean is taken for each minibatch)
train_rec_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample reconstruction loss
train_ccc_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample closest cluster center loss

optimizer = Adam(LR)



##### TRAINING PHASE 1 #####
for epoch in range(1, BEST_EPOCH_PHASE_1+1): 
    print(f"\nStart of epoch {epoch}")

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(batch_ds):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (nel latent space)

            # Compute the loss value for this minibatch
            loss_value = custom_loss(batch, decoded_batch, phase=1)
            
            train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
            train_rec_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, autoencoder.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

    print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
##### END OF TRAINING PHASE 1 #####
        


##### Find the n. of clusters with kmeans in the latent space
ds['miRNA_encoded'] = encoder(ds['miRNA_normalized'])   # (n_samples, ls_dim)
opt_silhouette = -1 

for n_clusters in np.arange(2, MAX_CLUSTERS+1):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_assignments = kmeans.fit_predict(ds['miRNA_encoded'])
    silhouette_avg = silhouette_score(ds['miRNA_encoded'], cluster_assignments)

    # Find the optimal number number of clusters, based on silhouette score
    if silhouette_avg >= opt_silhouette:    # better n. of clusters found
        n_clust_opt = n_clusters
        opt_silhouette = silhouette_avg

print(f"The best number of clusters found is: {n_clust_opt}")
n_clusters = n_clust_opt
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
cluster_assignments = kmeans.fit_predict(ds['miRNA_encoded'])

# Perform a 2D PCA to visualize the dataset
pca = PCA(2)
pc2 = pca.fit_transform(ds['miRNA_encoded'])
# Cluster centers in the latent space (needed for computing the loss in phase 2)
cluster_centers = kmeans.cluster_centers_
# Reduce cluster centers dimensions (for visualization)
kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

# Plot 2D visualization of the encoded dataset with the cluster assignments found
plot_2D_dataset(pc2, cluster_assignments, cluster_centers=kmeans.cluster_centers_, title='Encoded miRNA dataset')
print(f"Number of clusters: {cluster_centers.shape[0]}")

# Compute silhouette score in the latent space
silhouette_avg = silhouette_score(ds['miRNA_encoded'], cluster_assignments)
print(f"Silhouette on the encoded dataset: {silhouette_avg}")


       
##### TRAINING PHASE 2 #####
for epoch in range(BEST_EPOCH_PHASE_1 + 1, N_EPOCHS + 1):  # eventual early stopping
    print(f"\nStart of epoch {epoch}")

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(batch_ds):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (nel latent space)

            # Compute the loss value for this minibatch.
            # To compute this loss we need the cluster centers of the previous step (t-1)

            loss_value, l1, l2 = custom_loss(batch, decoded_batch, encoded_batch, cluster_centers, lmbd=LMBD, phase=2)
            
            train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
            train_rec_loss[epoch-1][step] = l1
            train_ccc_loss[epoch-1][step] = l2

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, autoencoder.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))


    ds['miRNA_encoded'] = encoder(ds['miRNA_normalized']) 

    # Perform KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_assignments_new = kmeans.fit_predict(ds['miRNA_encoded'])
    cluster_centers_new = np.copy(kmeans.cluster_centers_)

    if silhouette_avg < silhouette_score(ds['miRNA_encoded'], cluster_assignments_new):
        cluster_assignments = cluster_assignments_new
        cluster_centers = cluster_centers_new
        print("Cluster centers have changed")
            
    print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
    print("Mean training reconstruction loss for epoch %d = %.4f" % (epoch, train_rec_loss[epoch-1].mean()))
    print("Mean training closest cluster loss for epoch %d = %.4f" % (epoch, train_ccc_loss[epoch-1].mean()))
                
    silhouette_avg = silhouette_score(ds['miRNA_encoded'], cluster_assignments)
    print(f"Silhouette on the encoded dataset: {silhouette_avg}")

    # Perform a 2D PCA to visualize the dataset
    pca2 = PCA(2)
    pc2 = pca2.fit_transform(ds['miRNA_encoded'])
    # Plot the encoded space in 2D with predicted clusters and true cluster labels
    plot_2D_dataset(pc2, cluster_assignments, title='Encoded miRNA', caption='predicted clusters')
    plot_2D_dataset(pc2, true_cluster_labels, title='Encoded miRNA', caption='true clusters')

    # Plot confusion matrix
    plot_confusion_matrix(true_cluster_labels, cluster_assignments)
##### END OF TRAINING PHASE 2 #####

train_loss = train_loss.mean(axis=1)[:epoch-1]
train_rec_loss = train_rec_loss.mean(axis=1)[:epoch-1]
train_ccc_loss = train_ccc_loss.mean(axis=1)[:epoch-1]

# Evolution of training and validation losses
plt.plot(np.arange(1,len(train_loss)+1), train_loss)
plt.title('Train loss')
plt.show()

plt.plot(np.arange(1,len(train_rec_loss)+1), train_rec_loss)
plt.title('Train reconstruction loss')
plt.show()

plt.plot(np.arange(1,len(train_ccc_loss)+1), train_ccc_loss)
plt.title('Train closest cluster center loss')
plt.show()

# Save the autoencoder
autoencoder.save('autoencoder_miRNA_kidney.tf')


### meth

In [ ]:
n_samples, n_features = ds['meth'].shape
ds['meth'].shape

Grid search

In [ ]:
n_samples, n_features = ds['meth_normalized'].shape

##### Hyperparameters
# For the grid search
N_SPLITS = 3
kf = KFold(n_splits=N_SPLITS)
idx = np.arange(len(ds['meth_normalized']))   # 0,1,...,783; indices used for the K-fold splitting of the omic

bs_param = [16,32]
lr_param =[0.0005,0.0001]

# For the training
EPOCHS_PHASE_1 = 100    # max n. of epochs for the 1st phase (if no early stopping is taken)
EPOCHS_PHASE_2 = 60     # max n. of epochs for the 2nd phase (if no early stopping is taken)
N_EPOCHS = EPOCHS_PHASE_1 + EPOCHS_PHASE_2  # max n. of epochs (N_EPOCHS are performed if no early stopping is taken)
MIN_DELTA = 0.005   # threshold for the early stopping
PATIENCE = 30       # threshold for the early stopping
MAX_CLUSTERS = 10   # max n. of clusters to search for at the end of the 1st phase
LMBD = 0.1          # weight of the closest cluster center loss when it is summed to the reconstruction loss

optimal_val_rec_loss = np.inf   # this variable will record the best validation reconstruction loss obtained at the end of the grid search

for BATCH_SIZE, LR in cartesian(bs_param, lr_param):
    print('Training with:')
    print(f'Batch size: {BATCH_SIZE}')
    print(f'Learning Rate: {LR}')
    EARLY_STOPPING_PHASE_1 = []     # will store the N_SPLITS values obtained for the n. of epochs of phase 1 performed
    EARLY_STOPPING_PHASE_2 = []

    for train_index, test_index in kf.split(idx):
        # N_SPLITS-1 training splits, 1 validation split
        X_train, X_test = ds['meth_normalized'][train_index], ds['meth_normalized'][test_index]
        Y_train = true_cluster_labels[train_index]

        # Shuffle and split the training set in minibatches
        tensor_train_ds = tf.convert_to_tensor(X_train, dtype=tf.float32)    # cast to float32 (altrimenti non funziona, per un motivo a me ignoto)
        temp_ds = Dataset.from_tensor_slices(tensor_train_ds)
        shuffle_ds = temp_ds.shuffle(buffer_size=ds['meth'].shape[0], reshuffle_each_iteration=True)
        batch_ds = shuffle_ds.batch(BATCH_SIZE)

        # Build the validation set (no batches)
        tensor_val_ds = tf.convert_to_tensor(X_test, dtype=tf.float32)    # cast to float32 (altrimenti non funziona, per un motivo a me ignoto)

        

        #####################################################################################
        ##############################  MODEL ARCHITECTURE  #################################
        #####################################################################################

        ##### Autoencoder architecture
        input_window = Input(shape=n_features)

        # "encoded" is the encoded representation of the input
        x = Dense(2338, activation='softsign', name='encoder_1')(input_window)
        x = BatchNormalization()(x)
        x = Dense(234, activation='softsign', name='encoder_2')(x)
        x = BatchNormalization()(x)
        encoded_layer = Dense(64, activation='sigmoid', name='encoder_3')(x)

        # "decoded" is the lossy reconstruction of the input
        x = Dense(234, activation='softsign', name='decoder_1')(encoded_layer)
        x = BatchNormalization()(x)
        x = Dense(2338, activation='softsign', name='decoder_2')(x)
        x = BatchNormalization()(x)
        decoded_layer = Dense(n_features, activation='sigmoid', name='decoder_3')(x)

        # this model maps an input to its reconstruction and to its encoded representation in the latent space
        autoencoder = Model(input_window, outputs = [decoded_layer, encoded_layer])

        # this model maps an input to its encoded representation in the latent space
        encoder = Model(input_window, encoded_layer)  # restituisce solo il sample encoded

        print(autoencoder.summary())



        #####################################################################################
        ###################################  TRAINING  ######################################
        #####################################################################################
        
        train_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample loss (the mean is taken for each minibatch)
        train_rec_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample reconstruction loss
        train_ccc_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample closest cluster center loss

        val_rec_loss = np.zeros(N_EPOCHS)   # mean per-sample reconstruction loss
        
        optimizer = Adam(learning_rate = LR)





        ##### TRAINING PHASE 1 #####

        for epoch in range(1, EPOCHS_PHASE_1+1):  # eventual early stopping
            print(f"\nStart of epoch {epoch}")
            
            # Iterate over the batches of the dataset.
            for step, batch in enumerate(batch_ds):

                # Open a GradientTape to record the operations run
                # during the forward pass, which enables auto-differentiation.
                with tf.GradientTape() as tape:

                    # Run the forward pass of the layer.
                    # The operations that the layer applies
                    # to its inputs are going to be recorded
                    # on the GradientTape.
                    decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (in the latent space)

                    # Compute the loss value for this minibatch.
                    loss_value = custom_loss(batch, decoded_batch, phase=1)
                    
                    train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
                    train_rec_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)

                # Use the gradient tape to automatically retrieve
                # the gradients of the trainable variables with respect to the loss.
                grads = tape.gradient(loss_value, autoencoder.trainable_weights)

                # Run one step of gradient descent by updating
                # the value of the variables to minimize the loss.
                optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

            print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))


            # End of the epoch: VALIDATION STEP
            decoded_val_ds, _ = autoencoder(tensor_val_ds, training=False)  # Logits for this minibatch (in the latent space)
            loss_value_val = custom_loss(tensor_val_ds, decoded_val_ds, phase=1)    # mean per-sample validation reconstruction loss
            val_rec_loss[epoch-1] = tf.cast(loss_value_val, tf.float32)
            print("Mean validation loss for epoch %d = %.4f" % (epoch, val_rec_loss[epoch-1]))

            # Check if early stopping conditions are met
            stopEarly = Callback_EarlyStopping(val_rec_loss[:epoch-1], min_delta=MIN_DELTA, patience=PATIENCE)
            if stopEarly:
                print("Callback_EarlyStopping signal received at epoch= %d/%d"%(epoch,EPOCHS_PHASE_1))
                print("Terminating training")           
                break
        EARLY_STOPPING_PHASE_1.append(epoch) # populate vector of number of epochsof phase 1
        last_epoch_phase_1 = epoch  # less than or equal to EPOCHS_PHASE_1
        





        ##### Find the n. of clusters with kmeans in the latent space     
        ds['meth_encoded'] = encoder(X_train)   # (n_samples, ls_dim)
        opt_silhouette = -1 

        for n_clusters in np.arange(2, MAX_CLUSTERS+1):
            kmeans = KMeans(n_clusters=n_clusters, random_state=0)
            cluster_assignments = kmeans.fit_predict(ds['meth_encoded'])
            silhouette_avg = silhouette_score(ds['meth_encoded'], cluster_assignments)

            # Find the optimal number number of clusters, based on silhouette score
            if silhouette_avg >= opt_silhouette:    # better n. of clusters found
                n_clust_opt = n_clusters
                opt_silhouette = silhouette_avg

        print(f"The best number of clusters found is: {n_clust_opt}")
        n_clusters = n_clust_opt
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        cluster_assignments = kmeans.fit_predict(ds['meth_encoded'])

        # Perform a 2D PCA to visualize the dataset
        pca = PCA(2)
        pc2 = pca.fit_transform(ds['meth_encoded'])
        # Cluster centers in the latent space (needed for computing the loss in phase 2)
        cluster_centers = kmeans.cluster_centers_
        # Reduce cluster centers dimensions (for visualization)
        kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

        # Plot 2D visualization of the encoded dataset with the cluster assignments found
        plot_2D_dataset(pc2, cluster_assignments, cluster_centers=kmeans.cluster_centers_, title='Encoded meth dataset')
        print(f"Number of clusters: {cluster_centers.shape[0]}")

        # Compute silhouette score in the latent space
        silhouette_avg = silhouette_score(ds['meth_encoded'], cluster_assignments)
        print(f"Silhouette on the encoded dataset: {silhouette_avg}")







        ##### TRAINING PHASE 2 #####

        for epoch in range(last_epoch_phase_1 + 1, last_epoch_phase_1 + EPOCHS_PHASE_2 + 1):  # eventual early stopping
            print(f"\nStart of epoch {epoch}")

            # Iterate over the batches of the dataset.
            for step, batch in enumerate(batch_ds):

                # Open a GradientTape to record the operations run
                # during the forward pass, which enables auto-differentiation.
                with tf.GradientTape() as tape:

                    # Run the forward pass of the layer.
                    # The operations that the layer applies
                    # to its inputs are going to be recorded
                    # on the GradientTape.
                    decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (in the latent space)

                    # Compute the loss value for this minibatch.
                    # To compute this loss we need the cluster centers of the previous step (t-1)
                    loss_value, l1, l2 = custom_loss(batch, decoded_batch, encoded_batch, cluster_centers, lmbd=LMBD, phase=2)
                    
                    train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
                    train_rec_loss[epoch-1][step] = l1
                    train_ccc_loss[epoch-1][step] = l2

                # Use the gradient tape to automatically retrieve
                # the gradients of the trainable variables with respect to the loss.
                grads = tape.gradient(loss_value, autoencoder.trainable_weights)

                # Run one step of gradient descent by updating
                # the value of the variables to minimize the loss.
                optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

            # The autoencoder is now updated
            # Update the K-means cluster assignments and cluster centers
            ds['meth_encoded'] = encoder(X_train)
            kmeans = KMeans(n_clusters=n_clusters, random_state=0)
            cluster_assignments_new = kmeans.fit_predict(ds['meth_encoded'])
            cluster_centers_new = kmeans.cluster_centers_

            # Evaluate the silhouette w.r.t. the new cluster assignments:
            # if the encoded dataset is more separable with these assignments,
            # cluster assignment and cluster centers are updated, otherwise they remain unchanged
            if silhouette_avg < silhouette_score(ds['meth_encoded'], cluster_assignments_new):
                cluster_assignments = cluster_assignments_new
                cluster_centers = cluster_centers_new
                print("Cluster centers have changed")
                    
            print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
            print("Mean training reconstruction loss for epoch %d = %.4f" % (epoch, train_rec_loss[epoch-1].mean()))
            print("Mean training closest cluster loss for epoch %d = %.4f" % (epoch, train_ccc_loss[epoch-1].mean()))
            
            silhouette_avg = silhouette_score(ds['meth_encoded'], cluster_assignments)
            print(f"Silhouette on the encoded dataset: {silhouette_avg}")
            
            # Perform a 2D PCA to visualize the dataset
            pca = PCA(2)
            pc2 = pca.fit_transform(ds['meth_encoded'])
            # Plot the encoded space in 2D with predicted clusters and true cluster labels
            plot_2D_dataset(pc2, cluster_assignments, title='Encoded meth', caption='predicted clusters')
            plot_2D_dataset(pc2, Y_train, title='Encoded meth', caption='true clusters')
            # Plot confusion matrix
            plot_confusion_matrix(Y_train, cluster_assignments)


            # End of the epoch: VALIDATION STEP
            decoded_val_ds, _ = autoencoder(tensor_val_ds, training=False)  # Logits for this minibatch (in the latent space)
            loss_value_val = custom_loss(tensor_val_ds, decoded_val_ds, encoded_batch=None, phase=1)
            val_rec_loss[epoch-1] = tf.cast(loss_value_val, tf.float32)
            print("Mean validation loss for epoch %d = %.4f" % (epoch, val_rec_loss[epoch-1]))

            # Check if early stopping conditions are met
            stopEarly = Callback_EarlyStopping(val_rec_loss[EPOCHS_PHASE_1:epoch-1], min_delta=MIN_DELTA, patience=PATIENCE)
            if stopEarly:
                print("Callback_EarlyStopping signal received at epoch= %d/%d"%(epoch,EPOCHS_PHASE_1))
                print("Terminating training")
                break
        EARLY_STOPPING_PHASE_2.append(epoch - last_epoch_phase_1)   # populate vector of number of epochs of phase 2
    
    # Remove the zero values which have not been modified (because of early stopping)
    train_loss = train_loss.mean(axis=1)[:epoch-1]
    train_rec_loss = train_rec_loss.mean(axis=1)[:epoch-1]
    train_ccc_loss = train_ccc_loss.mean(axis=1)[:epoch-1]
    val_rec_loss = val_rec_loss[:epoch-1]

    plot_confusion_matrix(Y_train, cluster_assignments)

    # Evolution of training and validation losses
    plt.plot(np.arange(1,len(train_loss)+1), train_loss)
    plt.title('Train loss')
    plt.show()

    plt.plot(np.arange(1,len(train_rec_loss)+1), train_rec_loss)
    plt.title('Train reconstruction loss')
    plt.show()

    plt.plot(np.arange(1,len(train_ccc_loss)+1), train_ccc_loss)
    plt.title('Train closest cluster center loss')
    plt.show()

    plt.plot(np.arange(1,len(val_rec_loss)+1), val_rec_loss)
    plt.title('Validation reconstruction loss')
    plt.show()

    # For the grid search: are the current hyperparameters better than the others tested so far?
    if val_rec_loss[-1] < optimal_val_rec_loss:
        optimal_val_rec_loss = val_rec_loss[-1]
        BEST_BATCH_SIZE = BATCH_SIZE
        BEST_LR = LR
        BEST_EPOCH_PHASE_1 = int(np.trunc(np.mean(EARLY_STOPPING_PHASE_1))) # mean value of the epochs of the 1st phase (the average is across the N_SPLITS values obtained during cross-val)
        BEST_EPOCH_PHASE_2 = int(np.trunc(np.mean(EARLY_STOPPING_PHASE_2))) # mean value of the epochs of the 2nd phase (the average is across the N_SPLITS values obtained during cross-val)


print(f'\nBest params found: Batch size: {BEST_BATCH_SIZE}, Learning Rate: {BEST_LR}, Epochs 1st phase: {BEST_EPOCH_PHASE_1}, Epochs 2nd phase: {BEST_EPOCH_PHASE_2}')

BEST_BATCH_SIZE = 16

BEST_LR = 0.0001

BEST_EPOCH_PHASE_1 = 100

BEST_EPOCH_PHASE_2 = 60

Training

In [ ]:
# Hyperparameters
BATCH_SIZE = BEST_BATCH_SIZE
LR =  BEST_LR
N_EPOCHS = BEST_EPOCH_PHASE_1 + BEST_EPOCH_PHASE_2
EPOCHS_PHASE_1 = BEST_EPOCH_PHASE_1
MAX_CLUSTERS = 10   # max n. of clusters to search for at the end of the 1st phase
LMBD = 0.1 # 0.1


# Shuffle and split the dataset in minibatches
tensor_train_ds = tf.convert_to_tensor(ds['meth_normalized'], dtype=tf.float32)
temp_ds = Dataset.from_tensor_slices(tensor_train_ds)
shuffle_ds = temp_ds.shuffle(buffer_size=ds['meth'].shape[0], reshuffle_each_iteration=True)
batch_ds = shuffle_ds.batch(BATCH_SIZE)

#####################################################################################
##############################  MODEL ARCHITECTURE  #################################
#####################################################################################

##### Autoencoder architecture
# this is our input placeholder (cioè ogni sample in input deve avere n_features features)
input_window = Input(shape=n_features)

# "encoded" is the encoded representation of the input
x = Dense(2338, activation='softsign', name='encoder_1')(input_window)
x = BatchNormalization()(x)
x = Dense(234, activation='softsign', name='encoder_2')(x)
x = BatchNormalization()(x)
encoded_layer = Dense(64, activation='sigmoid', name='encoder_3')(x)

# "decoded" is the lossy reconstruction of the input
x = Dense(234, activation='softsign', name='decoder_1')(encoded_layer)
x = BatchNormalization()(x)
x = Dense(2338, activation='softsign', name='decoder_2')(x)
x = BatchNormalization()(x)
decoded_layer = Dense(n_features, activation='sigmoid', name='decoder_3')(x)

# this model maps an input to its reconstruction
autoencoder = Model(input_window, outputs = [decoded_layer,encoded_layer])  # quando è chiamato autoencoder.predict(), ritorna sia il sample encoded che quello decoded

# this model maps an input to its encoded representation in the latent space
encoder = Model(input_window, encoded_layer)  # restituisce solo il sample encoded

print(autoencoder.summary())


#####################################################################################
###################################  TRAINING  ######################################
#####################################################################################

train_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample loss (the mean is taken for each minibatch)
train_rec_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample reconstruction loss
train_ccc_loss = np.zeros((N_EPOCHS, len(batch_ds)))    # mean per-sample closest cluster center loss

optimizer = Adam(LR)



##### TRAINING PHASE 1 #####
for epoch in range(1, BEST_EPOCH_PHASE_1+1): 
    print(f"\nStart of epoch {epoch}")

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(batch_ds):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (nel latent space)

            # Compute the loss value for this minibatch
            loss_value = custom_loss(batch, decoded_batch, phase=1)
            
            train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
            train_rec_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, autoencoder.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))

    print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
##### END OF TRAINING PHASE 1 #####
        


##### Find the n. of clusters with kmeans in the latent space
ds['meth_encoded'] = encoder(ds['meth_normalized'])   # (n_samples, ls_dim)
opt_silhouette = -1 

for n_clusters in np.arange(2, MAX_CLUSTERS+1):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_assignments = kmeans.fit_predict(ds['meth_encoded'])
    silhouette_avg = silhouette_score(ds['meth_encoded'], cluster_assignments)

    # Find the optimal number number of clusters, based on silhouette score
    if silhouette_avg >= opt_silhouette:    # better n. of clusters found
        n_clust_opt = n_clusters
        opt_silhouette = silhouette_avg

print(f"The best number of clusters found is: {n_clust_opt}")
n_clusters = n_clust_opt
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
cluster_assignments = kmeans.fit_predict(ds['meth_encoded'])

# Perform a 2D PCA to visualize the dataset
pca = PCA(2)
pc2 = pca.fit_transform(ds['meth_encoded'])
# Cluster centers in the latent space (needed for computing the loss in phase 2)
cluster_centers = kmeans.cluster_centers_
# Reduce cluster centers dimensions (for visualization)
kmeans.cluster_centers_ = pca.transform(kmeans.cluster_centers_)

# Plot 2D visualization of the encoded dataset with the cluster assignments found
plot_2D_dataset(pc2, cluster_assignments, cluster_centers=kmeans.cluster_centers_, title='Encoded meth dataset')
print(f"Number of clusters: {cluster_centers.shape[0]}")

# Compute silhouette score in the latent space
silhouette_avg = silhouette_score(ds['meth_encoded'], cluster_assignments)
print(f"Silhouette on the encoded dataset: {silhouette_avg}")


       
##### TRAINING PHASE 2 #####
for epoch in range(BEST_EPOCH_PHASE_1 + 1, N_EPOCHS + 1):  # eventual early stopping
    print(f"\nStart of epoch {epoch}")

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(batch_ds):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            decoded_batch, encoded_batch = autoencoder(batch, training=True)  # Logits for this minibatch (nel latent space)

            # Compute the loss value for this minibatch.
            # To compute this loss we need the cluster centers of the previous step (t-1)

            loss_value, l1, l2 = custom_loss(batch, decoded_batch, encoded_batch, cluster_centers, lmbd=LMBD, phase=2)
            
            train_loss[epoch-1][step] = tf.cast(loss_value, tf.float32)
            train_rec_loss[epoch-1][step] = l1
            train_ccc_loss[epoch-1][step] = l2

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, autoencoder.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))


    ds['meth_encoded'] = encoder(ds['meth_normalized']) 

    # Perform KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_assignments_new = kmeans.fit_predict(ds['meth_encoded'])
    cluster_centers_new = np.copy(kmeans.cluster_centers_)

    if silhouette_avg < silhouette_score(ds['meth_encoded'], cluster_assignments_new):
        cluster_assignments = cluster_assignments_new
        cluster_centers = cluster_centers_new
        print("Cluster centers have changed")
            
    print("Mean training loss for epoch %d = %.4f" % (epoch, train_loss[epoch-1].mean()))
    print("Mean training reconstruction loss for epoch %d = %.4f" % (epoch, train_rec_loss[epoch-1].mean()))
    print("Mean training closest cluster loss for epoch %d = %.4f" % (epoch, train_ccc_loss[epoch-1].mean()))
                
    silhouette_avg = silhouette_score(ds['meth_encoded'], cluster_assignments)
    print(f"Silhouette on the encoded dataset: {silhouette_avg}")

    # Perform a 2D PCA to visualize the dataset
    pca2 = PCA(2)
    pc2 = pca2.fit_transform(ds['meth_encoded'])
    # Plot the encoded space in 2D with predicted clusters and true cluster labels
    plot_2D_dataset(pc2, cluster_assignments, title='Encoded meth', caption='predicted clusters')
    plot_2D_dataset(pc2, true_cluster_labels, title='Encoded meth', caption='true clusters')

    # Plot confusion matrix
    plot_confusion_matrix(true_cluster_labels, cluster_assignments)
##### END OF TRAINING PHASE 2 #####

train_loss = train_loss.mean(axis=1)[:epoch-1]
train_rec_loss = train_rec_loss.mean(axis=1)[:epoch-1]
train_ccc_loss = train_ccc_loss.mean(axis=1)[:epoch-1]

# Evolution of training and validation losses
plt.plot(np.arange(1,len(train_loss)+1), train_loss)
plt.title('Train loss')
plt.show()

plt.plot(np.arange(1,len(train_rec_loss)+1), train_rec_loss)
plt.title('Train reconstruction loss')
plt.show()

plt.plot(np.arange(1,len(train_ccc_loss)+1), train_ccc_loss)
plt.title('Train closest cluster center loss')
plt.show()

# Save the autoencoder
autoencoder.save('autoencoder_meth_kidney.tf')

## Save the autoencoders

In [ ]:
# Save autoencoders
!zip -r autoencoder_mRNA_kidney.zip autoencoder_mRNA_kidney.tf
!zip -r autoencoder_miRNA_kidney.zip autoencoder_miRNA_kidney.tf
!zip -r autoencoder_meth_kidney.zip autoencoder_meth_kidney.tf

## Integration

In [ ]:
#####
# Compute the distance matrix for each encoded omic (and then average them together)
dist_mat = []
for omic in omics:
    dist_mat.append(distance_matrix(ds[f'{omic}_encoded'], ds[f'{omic}_encoded']))
dist_mat = np.stack(dist_mat)


#####
# Cluster with spectral clustering
avg_dist_matrix_no_weights = np.average(dist_mat, axis=0)

embedding = MDS(n_components=2, dissimilarity='precomputed',random_state=0)
pc2_no_weights = embedding.fit_transform(avg_dist_matrix)
plot_2D_dataset(pc2_no_weights, true_cluster_labels, title='Avg distance matrix visualization', caption='True labels')

#####
# Results for the avg without weights integration
best_K_no_weights = 0
best_silh_no_weights = -1

MAX_CLUSTERS = 10
for K in range(2, MAX_CLUSTERS+1):
    ##### NO WEIGHTS
    spectral = SpectralClustering(n_clusters=K, affinity='precomputed_nearest_neighbors', n_neighbors=8) # random_state=0
    cluster_assignments = spectral.fit_predict(avg_dist_matrix_no_weights)
    silh_no_weights = silhouette_score(avg_dist_matrix_no_weights, cluster_assignments, metric="precomputed")
    if silh_no_weights > best_silh_no_weights:
        best_silh_no_weights = silh_no_weights
        best_K_no_weights = K

    # Visualize clustering results and conf mat
    print(f"----- {K} CLUSTERS, NO WEIGHTS -----")
    print(f"silhouette: {silh_no_weights}")
    print()
    print()

# Best result
print(f'Best K found (standard avg integration): {best_K_no_weights}')
spectral = SpectralClustering(n_clusters=best_K_no_weights, affinity='precomputed_nearest_neighbors', n_neighbors=8) # random_state=0
cluster_assignments = spectral.fit_predict(avg_dist_matrix_no_weights)
plot_2D_dataset(pc2_no_weights, cluster_assignments, title='Avg distance matrix', caption=f'{best_K_no_weights} clusters, silhouette = {best_silh_no_weights:.4f}')
plot_confusion_matrix(true_cluster_labels,cluster_assignments, title=f'Avg. distance matrix integration, K={best_K_no_weights}')
print()
print()
print()